# Pipeline ETL com os dados da Global Super Store

#### Link dos dados utilizados: https://www.kaggle.com/datasets/apoorvaappz/global-super-store-dataset
####

In [2]:
import pandas as pd
import numpy as np

## Carregamento dos dados

In [6]:
df_sales = pd.read_csv('Global_Superstore2.csv', encoding='latin-1')
df_sales.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,05-11-2013,06-11-2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [14]:
df_sales.shape

(51290, 24)

In [15]:
df_sales.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

#####
##### Como podemos ver temos algumas informações que podem ser divididas para outrass tabelas, como os dados do cliente e de produtos. Faremos isso abaixo
#####

## Transformação dos Dados

#### Aqui modelaremos os dados em um Star Schema e transformaremos antes de carregá-los em uma nuvem por exemplo

In [8]:
## Função para criação das tabelas dimensão
def create_dimension_tables(df):
    # Criação da tabela de dimensão Customer
    dim_customer = df[['Customer ID', 'Customer Name', 'Segment']].drop_duplicates().reset_index(drop=True)
    
    # Criação da tabela de dimensão Product
    dim_product = df[['Product ID', 'Product Name', 'Category', 'Sub-Category']].drop_duplicates().reset_index(drop=True)
    
    # Criação da tabela de dimensão Time
    dim_time = pd.DataFrame(pd.date_range(start=df['Order Date'].min(), end=df['Order Date'].max()), columns=['Date'])
    dim_time['Year'] = dim_time['Date'].dt.year
    dim_time['Month'] = dim_time['Date'].dt.month
    dim_time['Day'] = dim_time['Date'].dt.day
    dim_time['Weekday'] = dim_time['Date'].dt.weekday
    dim_time['Quarter'] = dim_time['Date'].dt.quarter

    # Criação da tabela de dimensão Ship Mode
    dim_ship_mode = df[['Ship Mode']].drop_duplicates().reset_index(drop=True)
    dim_ship_mode.index += 1  # Start indexing from 1
    dim_ship_mode.reset_index(inplace=True)
    dim_ship_mode.columns = ['Ship Mode ID', 'Ship Mode']

    return dim_customer, dim_product, dim_time, dim_ship_mode

dim_customer, dim_product, dim_time, dim_ship_mode = create_dimension_tables(df_sales)

In [9]:
dim_customer.head()

,Customer ID,Customer Name,Segment
0,RH-19495,Rick Hansen,Consumer
1,JR-16210,Justin Ritter,Corporate
2,CR-12730,Craig Reiter,Consumer
3,KM-16375,Katherine Murray,Home Office
4,RH-9495,Rick Hansen,Consumer


In [10]:
dim_product.head()

,Product ID,Product Name,Category,Sub-Category
0,TEC-AC-10003033,Plantronics CS510 - Over-the-Head monaural Wir...,Technology,Accessories
1,FUR-CH-10003950,"Novimex Executive Leather Armchair, Black",Furniture,Chairs
2,TEC-PH-10004664,"Nokia Smart Phone, with Caller ID",Technology,Phones
3,TEC-PH-10004583,"Motorola Smart Phone, Cordless",Technology,Phones
4,TEC-SHA-10000501,"Sharp Wireless Fax, High-Speed",Technology,Copiers


In [11]:
dim_time.head()

,Date,Year,Month,Day,Weekday,Quarter
0,2011-01-01,2011,1,1,5,1
1,2011-01-02,2011,1,2,6,1
2,2011-01-03,2011,1,3,0,1
3,2011-01-04,2011,1,4,1,1
4,2011-01-05,2011,1,5,2,1


In [13]:
dim_ship_mode

,Ship Mode ID,Ship Mode
0,1,Same Day
1,2,Second Class
2,3,First Class
3,4,Standard Class


In [18]:
## função para criar a tabela fato
def create_fact_table(df, dim_customer, dim_product, dim_time, dim_ship_mode):
    fact_sales = df.copy()
    
    # Mapeando IDs das dimensões
    fact_sales = fact_sales.merge(dim_customer, on=['Customer ID', 'Customer Name', 'Segment'])
    fact_sales = fact_sales.merge(dim_product, on=['Product ID', 'Product Name', 'Category', 'Sub-Category'])
    fact_sales['Order Date'] = pd.to_datetime(fact_sales['Order Date'], dayfirst=True)
    fact_sales['Ship Date'] = pd.to_datetime(fact_sales['Ship Date'], dayfirst=True)
    fact_sales['Order Date'] = fact_sales['Order Date'].dt.date
    fact_sales['Ship Date'] = fact_sales['Ship Date'].dt.date

    # Mapear Ship Mode ID
    fact_sales = fact_sales.merge(dim_ship_mode, on='Ship Mode')

    # Selecionar e renomear colunas para a tabela de fatos
    fact_sales = fact_sales[['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Sales', 'Quantity', 'Discount', 'Profit', 
                             'Customer ID', 'Product ID', 'Ship Mode ID']]
    
    return fact_sales

fact_sales = create_fact_table(df_sales, dim_customer, dim_product, dim_time, dim_ship_mode)

In [19]:
fact_sales.head()

,Row ID,Order ID,Order Date,Ship Date,Sales,Quantity,Discount,Profit,Customer ID,Product ID,Ship Mode ID
0,32298,CA-2012-124891,2012-07-31,2012-07-31,2309.650,7,0.0,762.1845,RH-19495,TEC-AC-10003033,1
1,26341,IN-2013-77878,2013-02-05,2013-02-07,3709.395,9,0.1,-288.7650,JR-16210,FUR-CH-10003950,2
2,25330,IN-2013-71249,2013-10-17,2013-10-18,5175.171,9,0.1,919.9710,CR-12730,TEC-PH-10004664,3
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,2892.510,5,0.1,-96.5400,KM-16375,TEC-PH-10004583,3
4,47221,SG-2013-4320,2013-11-05,2013-11-06,2832.960,8,0.0,311.5200,RH-9495,TEC-SHA-10000501,1


In [21]:
df_sales.shape

(51290, 24)

In [22]:
fact_sales.shape

(51290, 11)

## Carregamento dos Dados

#### Aqui o carregamento poderia ser em uma nuvem, mas por conveniência farei isso utilizando o duckdb, simulando um carregamento em um banco de dados

In [32]:
import duckdb

# Conectar ao DuckDB (ou criar um novo banco de dados local)
con = duckdb.connect('superstore.duckdb')

# Carregar Tabelas de Dimensão
con.execute("CREATE TABLE IF NOT EXISTS dim_customer AS SELECT * FROM dim_customer")
con.execute("CREATE TABLE IF NOT EXISTS dim_product AS SELECT * FROM dim_product")
con.execute("CREATE TABLE IF NOT EXISTS dim_time AS SELECT * FROM dim_time")
con.execute("CREATE TABLE IF NOT EXISTS dim_ship_mode AS SELECT * FROM dim_ship_mode")

# Carregar Tabela de Fatos
con.execute("CREATE TABLE IF NOT EXISTS fact_sales AS SELECT * FROM fact_sales")

# Para ver as tabelas criadas
print(con.execute("SHOW TABLES").fetchall())

[('dim_customer',), ('dim_product',), ('dim_ship_mode',), ('dim_time',), ('fact_sales',)]


In [33]:
## Fazendo testes
result = con.execute("SELECT * FROM fact_sales LIMIT 10").fetchdf()
print(result)

   Row ID         Order ID Order Date  Ship Date     Sales  Quantity  \
0   32298   CA-2012-124891 2012-07-31 2012-07-31  2309.650         7   
1   26341    IN-2013-77878 2013-02-05 2013-02-07  3709.395         9   
2   25330    IN-2013-71249 2013-10-17 2013-10-18  5175.171         9   
3   13524  ES-2013-1579342 2013-01-28 2013-01-30  2892.510         5   
4   47221     SG-2013-4320 2013-11-05 2013-11-06  2832.960         8   
5   22732    IN-2013-42360 2013-06-28 2013-07-01  2862.675         5   
6   30570    IN-2011-81826 2011-11-07 2011-11-09  1822.080         4   
7   31192    IN-2012-86369 2012-04-14 2012-04-18  5244.840         6   
8   40155   CA-2014-135909 2014-10-14 2014-10-21  5083.960         5   
9   40936   CA-2012-116638 2012-01-28 2012-01-31  4297.644        13   

   Discount     Profit Customer ID        Product ID  Ship Mode ID  
0       0.0   762.1845    RH-19495   TEC-AC-10003033             1  
1       0.1  -288.7650    JR-16210   FUR-CH-10003950             2  


In [35]:
products = con.execute("SELECT * FROM dim_product LIMIT 10").fetchdf()
print(products)

         Product ID                                       Product Name  \
0   TEC-AC-10003033  Plantronics CS510 - Over-the-Head monaural Wir...   
1   FUR-CH-10003950          Novimex Executive Leather Armchair, Black   
2   TEC-PH-10004664                  Nokia Smart Phone, with Caller ID   
3   TEC-PH-10004583                     Motorola Smart Phone, Cordless   
4  TEC-SHA-10000501                     Sharp Wireless Fax, High-Speed   
5   TEC-PH-10000030                Samsung Smart Phone, with Caller ID   
6   FUR-CH-10004050     Novimex Executive Leather Armchair, Adjustable   
7   FUR-TA-10002958       Chromcraft Conference Table, Fully Assembled   
8   OFF-BI-10003527  Fellowes PB500 Electric Punch Plastic Comb Bin...   
9   FUR-TA-10000198  Chromcraft Bull-Nose Wood Oval Conference Tabl...   

          Category Sub-Category  
0       Technology  Accessories  
1        Furniture       Chairs  
2       Technology       Phones  
3       Technology       Phones  
4       Technol

In [36]:
con.close()